In [49]:
import math
class ReliabilityCalculator:
    def __init__(self, system, n, tr, s, th, rr, ):
        self.mttfn = 1400000        
        self.n = n
        self.system = system
        self.tr = tr
        self.s = s # node total storage capacity
        self.th = th
        self.rr = rr
        
    def initDecStore(self, vdsCount=30, vdSize = 20000):
        self.vdsCount = vdsCount # in case of decstore - total count of vds in the system
        self.processingTime = 0.01 # ms - time to process one vd (decide where to put)
        self.vdSize = vdSize
        self.freeVDsCount = math.floor((self.s * self.n)/self.vdSize - vdsCount)# total free vds slots in the system
        print (self.freeVDsCount)
    
        
    def initFullBackup(self, b):
        self.b = b #number of times to visit getPANFBR function
        
    def getVDsInfo(self):
        return (self.vdsCount, self.freeVDsCount)
    
    def getTd(self, n):
        return self.vdsCount*self.processingTime/(n*3600)
        
    def getMTTRn(self, n=None): #MTTR node
        if self.system == "decStore":
            if self.tr == 0:
                mTTRn = (self.s /(3600*self.th*self.rr)) + self.tr
            else:
                mTTRn = (self.s /(3600*self.th*self.rr*n-1)) + self.getTd(n)
        else:
            mTTRn = (self.s /(3600*self.th*self.rr)) + self.tr
        return mTTRn
    
    def getMTTFn(self): #mTTF node
        return self.mttfn
    
    def getPANFBRs(self):
        pANFBER = 1
        if self.system == "raid5":
            pANFBER = self.getMTTRn()/(self.getMTTFn()/(self.n - 1))
        if self.system == "raid10":
            pANFBER = self.getMTTRn()/(self.getMTTFn())
        if self.system == "raid6":
            pANFBER = math.prod([(self.getMTTRn()*(self.n - i))/self.getMTTFn() for i in range (1,3)])#/2
        if self.system == "decStore":
            marc = self.getMARC()
            pANFBER = (((marc*self.getMTTRn())/self.getMTTFn())**marc) * (self.getMTTFn()/math.prod([self.n-i for i in range(1, marc+1)]))
        if self.system == "fullbackup":
            pANFBER = math.prod([(self.getMTTRn()*(self.b+1-1))/self.getMTTFn() for i in range(1, self.b)])
        return pANFBER
    
    def getMTTFs(self): #MTTF system
        MTTFs = ((self.getMTTFn())/self.n)* (1/self.getPANFBRs())
        return MTTFs
    def F(self, n):
        if n == 1:
            return self.freeVDsCount
        return self.F(n-1) - (self.vdsCount/(self.n-n)) - (self.F(n-1)/(self.n-n))
    def getMARC(self):
        for i in range (1, self.n):
            if self.F(i) < 0:
                return i-1
        
        

In [3]:

print([i for i in range(1,3)])

[1, 2]


In [2]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
pANFBER: 74.07407407407408 2000000 5
1799999999.9999995


In [3]:
#raid5
system = ReliabilityCalculator("raid5", 6, 0, 2000000, 15, 0.5)
print(system.getMTTRn())
print(system.getMTTFs())

74.07407407407408
pANFBER: 74.07407407407408 2000000 5
1799999999.9999995


In [11]:
#raid10
system = ReliabilityCalculator("raid10", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())
print(system.getMTTFn(), system.getMTTRn(), system.getMTTFn())

185.1851851851852
3599999999.999999
2000000 185.1851851851852 2000000


In [15]:
#raid6
system = ReliabilityCalculator("raid6", 6, 0, 2000000, 15, 0.2)
print(system.getMTTRn())
print(system.getMTTFs())

185.1851851851852
666791999999.9999


In [60]:
#decStore
total = 2000000
n = 4
usage = 70 #%
vdSize = 20000
vdsCount = math.floor(total * n * usage / (100*20000))
print (vdsCount)
system = ReliabilityCalculator("decStore", n, 0, total, 100, 0.2)
system.initDecStore(vdsCount=vdsCount)
print(system.getMTTRn())
print(system.getMTTFs())
print(system.getMARC())

280
120
27.77777777777778
37800.0
1


In [61]:
#decStore
system = ReliabilityCalculator("fullbackup", 4, 48, 2000000, 100, 0.2)
system.initFullBackup(2)
print(system.getMTTRn())
print(system.getMTTFs())

75.77777777777777
3233137829.9120235
